In [68]:
import numpy as np
import yfinance as yf
import pandas as pd
import methods.selenium_patch as selenium_patch
from datetime import timedelta
from methods.scraper import Ticker

In [69]:
ticker = "TRMD"
data = Ticker(ticker).macrotrends_key_financial_ratios(frequency="quarterly")
display(data)

field_name,Current Ratio,Long-term Debt / Capital,Debt/Equity Ratio,Gross Margin,Operating Margin,EBIT Margin,EBITDA Margin,Pre-Tax Profit Margin,Net Profit Margin,Asset Turnover,Inventory Turnover Ratio,Receiveable Turnover,Days Sales In Receivables,ROE - Return On Equity,Return On Tangible Equity,ROA - Return On Assets,ROI - Return On Investment,Book Value Per Share,Operating Cash Flow Per Share,Free Cash Flow Per Share
2025-09-30,2.36500,0.26530,0.41780,82.66200,28.51720,28.51720,,23.00060,22.65030,0.10710,0.91380,1.63850,54.93000,3.60480,3.61180,2.42670,2.64830,21.96630,1.1863,1.1585
2025-06-30,2.56600,0.31520,0.53430,79.60030,21.95430,21.95430,,19.16240,18.59140,0.09280,0.92520,1.56190,57.62060,2.78560,2.79100,1.72810,1.90760,21.50310,1.5994,1.6397
2025-03-31,2.55030,0.32280,0.55190,79.82380,22.15130,22.15130,,20.75360,18.93040,0.09440,0.91590,1.43710,62.62530,2.93620,2.94200,1.80400,1.98860,21.90390,0.82520,1.17780
2024-12-31,2.15590,0.33840,0.59100,79.33180,25.15560,25.15560,,24.50050,25.48310,0.08800,0.92250,1.25380,71.78190,3.73050,3.73710,2.23080,2.46830,21.32370,1.2812,-0.0597
2024-09-30,2.47360,0.33820,0.59370,83.14970,35.25930,35.25930,,35.15180,35.07120,0.10900,0.97510,1.69060,53.23570,6.40120,6.41380,3.82770,4.23610,21.44750,2.4456,0.8988
2024-06-30,3.18350,0.34640,0.61560,85.67510,44.27690,44.27690,,44.11700,44.43680,0.12630,0.90350,1.51190,59.52710,9.50240,9.52100,5.60380,6.21100,21.74150,2.8438,2.9319
2024-03-31,3.62810,0.35220,0.63230,87.25510,46.31840,46.31840,,47.46680,47.21910,0.12660,0.80400,1.37880,65.27580,10.13080,10.14890,5.96390,6.56230,22.99550,2.01510,0.12310
2023-12-31,2.60440,0.34740,0.63600,85.79160,40.07220,40.07220,,47.65340,47.60190,0.13510,0.89300,1.42840,63.00930,11.08640,11.11040,6.43530,7.23490,19.32130,2.3151,2.0478
2023-09-30,2.33200,0.36290,0.69340,84.86460,37.14050,37.14050,,35.66040,34.71100,0.12750,0.76770,1.22220,73.63870,7.92170,7.93890,4.42650,5.04690,18.59120,2.272,2.2429
2023-06-30,1.78570,0.35340,0.73660,85.74030,51.07990,51.07990,,47.98340,48.34760,0.13430,0.81910,1.16770,77.07260,11.83680,11.86410,6.48120,7.65330,18.55810,2.1778,1.0393


In [70]:
prices = yf.Ticker(ticker).history("max").get("Close")
earning_prices = []
for date in data.index:
    got_price = False
    i = 0
    while got_price == False and i<7:
        try:
            earning_prices.append(prices[pd.to_datetime(date).tz_localize("America/New_York") - timedelta(days=i)]) # type: ignore
            got_price = True
        except:
            i += 1
    if got_price == False:
        earning_prices.append(np.nan)

earning_changes = [np.nan]
for i in range(1, len(earning_prices)):
    earning_changes.append((earning_prices[i-1]/earning_prices[i]-1)*100)

In [71]:
data["Future Change%"] = earning_changes
data["Price"] = earning_prices
data["Ticker"] = ticker
data = data[data.columns.to_list()[::-1]]
data

field_name,Ticker,Price,Future Change%,Free Cash Flow Per Share,Operating Cash Flow Per Share,Book Value Per Share,ROI - Return On Investment,ROA - Return On Assets,Return On Tangible Equity,ROE - Return On Equity,...,Asset Turnover,Net Profit Margin,Pre-Tax Profit Margin,EBITDA Margin,EBIT Margin,Operating Margin,Gross Margin,Debt/Equity Ratio,Long-term Debt / Capital,Current Ratio
2025-09-30,TRMD,20.067520,NaN,1.1585,1.1863,21.96630,2.64830,2.42670,3.61180,3.60480,...,0.10710,22.65030,23.00060,,28.51720,28.51720,82.66200,0.41780,0.26530,2.36500
2025-06-30,TRMD,15.898335,26.224040,1.6397,1.5994,21.50310,1.90760,1.72810,2.79100,2.78560,...,0.09280,18.59140,19.16240,,21.95430,21.95430,79.60030,0.53430,0.31520,2.56600
2025-03-31,TRMD,15.381306,3.361410,1.17780,0.82520,21.90390,1.98860,1.80400,2.94200,2.93620,...,0.09440,18.93040,20.75360,,22.15130,22.15130,79.82380,0.55190,0.32280,2.55030
2024-12-31,TRMD,17.591860,-12.565778,-0.0597,1.2812,21.32370,2.46830,2.23080,3.73710,3.73050,...,0.08800,25.48310,24.50050,,25.15560,25.15560,79.33180,0.59100,0.33840,2.15590
2024-09-30,TRMD,29.375561,-40.113961,0.8988,2.4456,21.44750,4.23610,3.82770,6.41380,6.40120,...,0.10900,35.07120,35.15180,,35.25930,35.25930,83.14970,0.59370,0.33820,2.47360
2024-06-30,TRMD,31.757290,-7.499787,2.9319,2.8438,21.74150,6.21100,5.60380,9.52100,9.50240,...,0.12630,44.43680,44.11700,,44.27690,44.27690,85.67510,0.61560,0.34640,3.18350
2024-03-31,TRMD,26.445358,20.086442,0.12310,2.01510,22.99550,6.56230,5.96390,10.14890,10.13080,...,0.12660,47.21910,47.46680,,46.31840,46.31840,87.25510,0.63230,0.35220,3.62810
2023-12-31,TRMD,23.010105,14.929324,2.0478,2.3151,19.32130,7.23490,6.43530,11.11040,11.08640,...,0.13510,47.60190,47.65340,,40.07220,40.07220,85.79160,0.63600,0.34740,2.60440
2023-09-30,TRMD,19.922417,15.498564,2.2429,2.272,18.59120,5.04690,4.42650,7.93890,7.92170,...,0.12750,34.71100,35.66040,,37.14050,37.14050,84.86460,0.69340,0.36290,2.33200
2023-06-30,TRMD,16.502264,20.725354,1.0393,2.1778,18.55810,7.65330,6.48120,11.86410,11.83680,...,0.13430,48.34760,47.98340,,51.07990,51.07990,85.74030,0.73660,0.35340,1.78570


In [ ]:
def imputer(df: pd.DataFrame) -> pd.DataFrame:
    numerical_columns = df.columns.drop("Ticker")
    max_nans_share = 0.25
    # Go through all data points and remove rows with too many nans
    for row in df.index:
        row_nans = 0
        for column in numerical_columns:
            val = df.loc[row, column]
            if val == '':
                val = np.nan
                df.loc[row, column] = np.nan
            if np.isnan(float(val)): # type: ignore
                row_nans += 1
        if row_nans/len(numerical_columns) > max_nans_share or np.isnan(df.loc[row, "Price"]): # type: ignore
            df = df.drop(row)
    
    # Go through all data points and remove columns with too many nans
    for column in numerical_columns:
        col_nans = 0
        for row in df.index:
            val = df.loc[row, column]
            if np.isnan(float(val)): # type: ignore
                col_nans += 1
        if col_nans/len(numerical_columns) > max_nans_share:
            df = df.drop(column, axis=1)
        
    return df

imputer(data)

field_name,Ticker,Price,Future Change%,Free Cash Flow Per Share,Operating Cash Flow Per Share,Book Value Per Share,ROI - Return On Investment,ROA - Return On Assets,Return On Tangible Equity,ROE - Return On Equity,...,Inventory Turnover Ratio,Asset Turnover,Net Profit Margin,Pre-Tax Profit Margin,EBIT Margin,Operating Margin,Gross Margin,Debt/Equity Ratio,Long-term Debt / Capital,Current Ratio
2025-09-30,TRMD,20.067520,NaN,1.1585,1.1863,21.96630,2.64830,2.42670,3.61180,3.60480,...,0.91380,0.10710,22.65030,23.00060,28.51720,28.51720,82.66200,0.41780,0.26530,2.36500
2025-06-30,TRMD,15.898335,26.224040,1.6397,1.5994,21.50310,1.90760,1.72810,2.79100,2.78560,...,0.92520,0.09280,18.59140,19.16240,21.95430,21.95430,79.60030,0.53430,0.31520,2.56600
2025-03-31,TRMD,15.381306,3.361410,1.17780,0.82520,21.90390,1.98860,1.80400,2.94200,2.93620,...,0.91590,0.09440,18.93040,20.75360,22.15130,22.15130,79.82380,0.55190,0.32280,2.55030
2024-12-31,TRMD,17.591860,-12.565778,-0.0597,1.2812,21.32370,2.46830,2.23080,3.73710,3.73050,...,0.92250,0.08800,25.48310,24.50050,25.15560,25.15560,79.33180,0.59100,0.33840,2.15590
2024-09-30,TRMD,29.375561,-40.113961,0.8988,2.4456,21.44750,4.23610,3.82770,6.41380,6.40120,...,0.97510,0.10900,35.07120,35.15180,35.25930,35.25930,83.14970,0.59370,0.33820,2.47360
2024-06-30,TRMD,31.757290,-7.499787,2.9319,2.8438,21.74150,6.21100,5.60380,9.52100,9.50240,...,0.90350,0.12630,44.43680,44.11700,44.27690,44.27690,85.67510,0.61560,0.34640,3.18350
2024-03-31,TRMD,26.445358,20.086442,0.12310,2.01510,22.99550,6.56230,5.96390,10.14890,10.13080,...,0.80400,0.12660,47.21910,47.46680,46.31840,46.31840,87.25510,0.63230,0.35220,3.62810
2023-12-31,TRMD,23.010105,14.929324,2.0478,2.3151,19.32130,7.23490,6.43530,11.11040,11.08640,...,0.89300,0.13510,47.60190,47.65340,40.07220,40.07220,85.79160,0.63600,0.34740,2.60440
2023-09-30,TRMD,19.922417,15.498564,2.2429,2.272,18.59120,5.04690,4.42650,7.93890,7.92170,...,0.76770,0.12750,34.71100,35.66040,37.14050,37.14050,84.86460,0.69340,0.36290,2.33200
2023-06-30,TRMD,16.502264,20.725354,1.0393,2.1778,18.55810,7.65330,6.48120,11.86410,11.83680,...,0.81910,0.13430,48.34760,47.98340,51.07990,51.07990,85.74030,0.73660,0.35340,1.78570
